<a href="https://www.kaggle.com/code/ramezsamy/detect-spam-emails?scriptVersionId=185120704" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# importing needed libiraies

In [1]:
#library for storing data
import pandas as pd
import numpy as np
#Libraries for ploting
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px 
#import warnings to avoid any error
import warnings
warnings.filterwarnings('ignore')

print("importing done")

importing done


# reading and explore our data

In [2]:
df = pd.read_csv("/kaggle/input/spam-email/spam.csv")


In [3]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
df.shape

(5572, 2)

In [6]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

# preprocessing to make our data ready to fit it in model

In [7]:
len(df[df.Category == "ham"]) # ham = not spam 

4825

In [8]:
len(df[df.Category == "spam"])

747

In [9]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [10]:
df["spam"] = df["Category"].replace(to_replace=["ham", "spam"], value=[0,1])

In [11]:
df

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [12]:
df['spam']= df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


# split our data by using train train_test_split

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.Message,df.spam, train_size=.8)

In [14]:
y_train

995     0
2331    0
5314    1
5505    0
3038    0
       ..
4840    0
956     0
3872    0
90      0
918     0
Name: spam, Length: 4457, dtype: int64

In [15]:
len(X_train[:1].to_list()[0].split())

7

In [16]:

from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[0].shape


(7715,)

In [17]:
X_train_count

<4457x7715 sparse matrix of type '<class 'numpy.int64'>'
	with 59241 stored elements in Compressed Sparse Row format>

# we ill use naive_bayes algorithm

In [18]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y_train)

MultinomialNB()

# check CountVectorizer function

In [19]:
emails = [
    'Hey Mohamed, can we get together to watch football game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1])

# our model accuracy

In [20]:
X_test_count = v.transform(X_test)
model.score(X_test_count, y_test)

0.9910313901345291

In [21]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [22]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [23]:
clf.score(X_test,y_test)

0.9910313901345291

In [24]:
clf.predict(emails)

array([0, 1])